In [16]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


BATCH_SIZE = 32

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_loader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [17]:
from torch.nn import Linear
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Softmax
from torch.nn import functional

class MNISTModel(torch.nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = Conv2d(1, 5, kernel_size=3,stride=1)
        self.maxpool1 = MaxPool2d(kernel_size=2)
        self.flatten = torch.nn.Flatten()
        self.linear1 = Linear(in_features=5*13*13, out_features=13*13)
        self.linear2 = Linear(13*13, 10)
        
    def forward(self, input):
        t = self.conv1(input)
        t = self.maxpool1(t)
        t = self.flatten(t)
        t = self.linear1(t)
        t = self.linear2(functional.relu(t))
        return t

In [18]:
model = MNISTModel()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

def train_loop(dataloader, model, optimizer, loss_fn):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * BATCH_SIZE + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    correct = 0
    loss = 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            loss += loss_fn(pred, y).type(torch.float).item()


    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    accuracy = correct / size
    loss = loss / num_batches

    print(f"test result: acc: {accuracy*100:>2f} avg_loss: {loss:>2f}")

epochs = 10
for t in range(epochs):
    print(f"Epoch: {t+1}")
    print("-----------------------------------------")
    train_loop(train_loader, model, optimizer, loss_fn)
    test_loop(test_loader, model, loss_fn)

print("Done!")

Epoch: 1
-----------------------------------------
loss: 2.298599  [   32/60000]
loss: 2.135046  [32032/60000]
test result: acc: 60.520000 avg_loss: 1.680220
Epoch: 2
-----------------------------------------
loss: 1.682949  [   32/60000]
loss: 1.080475  [32032/60000]
test result: acc: 71.520000 avg_loss: 0.863712
Epoch: 3
-----------------------------------------
loss: 0.811738  [   32/60000]
loss: 0.548570  [32032/60000]
test result: acc: 74.750000 avg_loss: 0.706439
Epoch: 4
-----------------------------------------
loss: 0.663366  [   32/60000]
loss: 0.714719  [32032/60000]
test result: acc: 77.290000 avg_loss: 0.639320
Epoch: 5
-----------------------------------------
loss: 0.722729  [   32/60000]
loss: 0.503816  [32032/60000]
test result: acc: 78.220000 avg_loss: 0.600728
Epoch: 6
-----------------------------------------
loss: 0.707766  [   32/60000]
loss: 0.375713  [32032/60000]
test result: acc: 79.510000 avg_loss: 0.570682
Epoch: 7
-----------------------------------------
l